In [1]:
import os
import psycopg2
import pandas as pd
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel
from pymilvus import MilvusClient

# ================== Config ==================
DB_PARAMS = {
    "dbname": "video_frame",   # database đã tạo
    "user": "postgres",
    "password": "123",         # đổi theo mật khẩu của bạn
    "host": "localhost",
    "port": "5432"
}

CSV_DIR   = r"D:\Big_project_2025\Video_Similarity_Search\data\csv"
FRAME_DIR = r"D:\Big_project_2025\Video_Similarity_Search\data\key_frame"
VIDEO_DIR = r"D:\Big_project_2025\Video_Similarity_Search\data\video"
MODEL_DIR = r"D:\Big_project_2025\huggingface_cache"  # nơi lưu model CLIP

# ================== Kết nối DB + Milvus ==================s
conn = psycopg2.connect(**DB_PARAMS)
cur = conn.cursor()
client = MilvusClient(uri="http://localhost:19530")


d:\Big_project_2025\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.27.2 is exactly one major version older than the runtime version 6.31.1 at schema.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
d:\Big_project_2025\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.27.2 is exactly one major version older than the runtime version 6.31.1 at common.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
d:\Big_project_2025\venv\Lib\site-packages\google\protobuf\runtime_version.py:98: UserWarning: Protobuf gencode version 5.27.2 is exactly one major version older than the runtime version 6.31.1 at milvus.proto. Please update the gencode to avoid compatibility violations in the next runtime release.
  warnings.warn(
d:\Big_project_2025\venv\Lib\site-packages\google

# Cell 2. Tạo bảng PostgreSQL (chạy 1 lần)
chạy bên pg admin rồi

# Cell 3 – Tạo collection Milvus (chạy 1 lần)

In [9]:
# Kết nối Milvus
milvus_client = MilvusClient(uri="http://localhost:19530")

# Tên collection
collection_name = "text_image_video_collection"

# Xóa collection cũ (nếu có) -> khởi tạo lại từ đầu
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)
    print(f"⚠️ Collection '{collection_name}' đã bị xóa")

# Tạo collection mới
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=512,              # Vector size của CLIP ViT-B/32
    auto_id=True,               # Tự động tạo ID
    enable_dynamic_field=True   # Cho phép thêm field động (vd: frame_path)
)

# Kiểm tra danh sách collection hiện có
collections = milvus_client.list_collections()
print("✅ Danh sách collection:", collections)


✅ Danh sách collection: ['video_search', 'image_collection', 'text_image_video_collection', 'Movies', 'my_rag_collection']


In [3]:
# Mô tả Bộ sưu tập
res = client.describe_collection(
    collection_name="text_image_video_collection"
)

res

{'collection_name': 'text_image_video_collection',
 'auto_id': True,
 'num_shards': 1,
 'description': '',
 'fields': [{'field_id': 100,
   'name': 'id',
   'description': '',
   'type': <DataType.INT64: 5>,
   'params': {},
   'auto_id': True,
   'is_primary': True},
  {'field_id': 101,
   'name': 'vector',
   'description': '',
   'type': <DataType.FLOAT_VECTOR: 101>,
   'params': {'dim': 512}}],
 'functions': [],
 'aliases': [],
 'collection_id': 460239432247376457,
 'consistency_level': 2,
 'properties': {},
 'num_partitions': 1,
 'enable_dynamic_field': True,
 'created_timestamp': 460240111572615174}

# Cell 4 – Load CLIP model

In [2]:

import torch
from PIL import Image
from transformers import CLIPModel, CLIPProcessor

# Xác định thiết bị sử dụng: ưu tiên GPU (cuda) nếu có, nếu không thì dùng CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Tải mô hình CLIP đã được huấn luyện trước từ Hugging Face, lưu trữ tại thư mục MODEL_DIR
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32", cache_dir=MODEL_DIR).to(device)

# Tải bộ xử lý (processor) của CLIP để chuẩn bị dữ liệu đầu vào (hình ảnh, văn bản)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32", cache_dir=MODEL_DIR)

def encode_image(image_path):
    """
    Hàm mã hóa hình ảnh thành vector đặc trưng sử dụng mô hình CLIP.
    
    Args:
        image_path (str): Đường dẫn tới file hình ảnh cần mã hóa.
        
    Returns:
        numpy.ndarray: Vector đặc trưng của hình ảnh hoặc None nếu có lỗi.
    """
    try:
        # Mở file hình ảnh bằng PIL
        image = Image.open(image_path)
        
        # Tiền xử lý hình ảnh: chuyển đổi định dạng, chuẩn hóa, và đưa vào tensor PyTorch
        inputs = processor(images=image, return_tensors="pt").to(device)
        
        # Tắt tính toán gradient để tăng tốc độ và giảm sử dụng bộ nhớ
        with torch.no_grad():
            # Lấy vector đặc trưng của hình ảnh từ mô hình CLIP
            image_features = model.get_image_features(**inputs)
        
        # Chuyển vector đặc trưng từ tensor về mảng numpy và trả về
        return image_features[0].cpu().numpy()
    
    except Exception as e:
        # In thông báo lỗi nếu quá trình xử lý hình ảnh thất bại
        print(f"Lỗi khi xử lý {image_path}: {e}")
        return None

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


# Cell 5 – Hàm insert tách riêng

In [ ]:


def insert_to_milvus(frame_path, emb):
    """
    Chèn vector đặc trưng và đường dẫn frame vào collection trong Milvus.

    Args:
        frame_path (str): Đường dẫn tới file frame (hình ảnh).
        emb (numpy.ndarray): Vector đặc trưng của frame (thường từ mô hình như CLIP).

    Returns:
        str: ID của bản ghi vừa chèn trong Milvus.
    """
    # Sử dụng client của Milvus để chèn dữ liệu vào collection có tên "text_image_video_collection"
    res = client.insert(
        collection_name="text_image_video_collection",
        data=[{"vector": emb, "frame_path": frame_path}]
    )
    # Trả về ID đầu tiên của bản ghi vừa chèn
    return res["ids"][0]


def insert_to_postgres(video_id, frame_path, pts_time, frame_idx, fps, milvus_id):
    """
    Chèn thông tin frame vào bảng frame_mappings trong PostgreSQL.

    Args:
        video_id (str): ID của video chứa frame.
        frame_path (str): Đường dẫn tới file frame.
        pts_time (float): Thời gian trình bày (presentation timestamp) của frame trong video.
        frame_idx (int): Chỉ số của frame trong video.
        fps (float): Tốc độ khung hình (frames per second) của video.
        milvus_id (str): ID của bản ghi tương ứng trong Milvus.

    Returns:
        None
    """
    # Thực hiện câu lệnh SQL để chèn dữ liệu vào bảng frame_mappings
    cur.execute("""
        INSERT INTO frame_mappings (video_id, frame_path, pts_time, frame_idx, fps, milvus_id)
        VALUES (%s, %s, %s, %s, %s, %s)
        ON CONFLICT (frame_path) DO NOTHING
    """, (video_id, frame_path, pts_time, frame_idx, fps, milvus_id))

# Cell 6 – Index toàn bộ video (dùng khi mới chạy lần đầu hoặc re-index)
- Pipeline này xây dựng cơ sở dữ liệu cho hệ thống tìm kiếm video dựa trên nội dung hình ảnh.
- Milvus lưu vector để tìm kiếm tương đồng (similarity search).
- PostgreSQL lưu metadata để truy vấn thông tin chi tiết (ví dụ: lấy frame tại thời điểm cụ thể trong video).

In [ ]:

# ================== Cell 6: Index dữ liệu ==================
def insert_to_milvus(frame_path, emb):
    """
    Chèn vector đặc trưng và đường dẫn frame vào collection Milvus.

    Args:
        frame_path (str): Đường dẫn tới file frame (hình ảnh).
        emb (numpy.ndarray): Vector đặc trưng của frame từ mô hình CLIP.

    Returns:
        str or None: ID của bản ghi trong Milvus hoặc None nếu lỗi.
    """
    try:
        # Gọi API client của Milvus để chèn dữ liệu vào collection "text_image_video_collection"
        # Dữ liệu là một danh sách chứa dictionary với hai trường: vector (đặc trưng) và frame_path
        res = client.insert(
            collection_name="text_image_video_collection",
            data=[{"vector": emb, "frame_path": frame_path}]
        )
        # Kiểm tra kết quả trả về: Milvus v2 trả về dict với trường "ids" chứa danh sách ID
        # Lấy ID đầu tiên nếu tồn tại, nếu không trả về None
        return res["ids"][0] if "ids" in res and len(res["ids"]) > 0 else None
    except Exception as e:
        # In lỗi chi tiết nếu chèn thất bại (ví dụ: kết nối Milvus lỗi, schema không khớp)
        print(f"❌ Lỗi insert Milvus cho {frame_path}: {e}")
        return None

def insert_to_postgres(video_id, frame_path, pts_time, frame_idx, fps, milvus_id):
    """
    Chèn metadata của frame vào bảng frame_mappings trong PostgreSQL.

    Args:
        video_id (str): ID của video chứa frame.
        frame_path (str): Đường dẫn tới file frame.
        pts_time (float): Thời gian trình bày (presentation timestamp) của frame.
        frame_idx (int): Chỉ số thứ tự của frame trong video.
        fps (float): Tốc độ khung hình của video.
        milvus_id (str): ID của bản ghi tương ứng trong Milvus.

    Returns:
        None
    """
    try:
        # Thực hiện câu lệnh SQL INSERT để thêm bản ghi vào bảng frame_mappings
        # ON CONFLICT (frame_path) DO NOTHING: Bỏ qua nếu frame_path đã tồn tại
        cur.execute("""
            INSERT INTO frame_mappings (video_id, frame_path, pts_time, frame_idx, fps, milvus_id)
            VALUES (%s, %s, %s, %s, %s, %s)
            ON CONFLICT (frame_path) DO NOTHING
        """, (video_id, frame_path, pts_time, frame_idx, fps, milvus_id))
    except Exception as e:
        # In lỗi nếu chèn thất bại (ví dụ: kết nối DB lỗi, kiểu dữ liệu không khớp)
        print(f"❌ Lỗi insert Postgres cho {frame_path}: {e}")

def index_videos():
    """
    Pipeline lập chỉ mục video: mã hóa frame, lưu vector vào Milvus và metadata vào PostgreSQL.
    Duyệt qua tất cả video trong thư mục VIDEO_DIR, xử lý frame và file CSV tương ứng.
    """
    # Duyệt qua các thư mục con trong FRAME_DIR (mỗi thư mục chứa frame của một video)
    for key_frame_dir in os.listdir(FRAME_DIR):
        # Tạo đường dẫn đầy đủ tới thư mục frame, file CSV và video
        frame_dir_path = os.path.join(FRAME_DIR, key_frame_dir)
        csv_path = os.path.join(CSV_DIR, f"{key_frame_dir}.csv")
        video_path = os.path.join(VIDEO_DIR, f"{key_frame_dir}.mp4")

        # Kiểm tra sự tồn tại của thư mục frame, file video và file CSV
        if not (os.path.isdir(frame_dir_path) and os.path.exists(video_path) and os.path.exists(csv_path)):
            print(f"⚠️ Thiếu file cho {key_frame_dir}, bỏ qua.")
            continue

        # Đọc file CSV chứa thông tin mapping (pts_time, fps) của frame
        mapping_df = pd.read_csv(csv_path)

        # Chèn thông tin video vào bảng videos hoặc lấy video_id nếu video đã tồn tại
        # Đảm bảo đường dẫn sử dụng dấu "/" để tương thích đa nền tảng
        cur.execute("""
            INSERT INTO videos (video_path, title, description)
            VALUES (%s, %s, %s)
            ON CONFLICT (video_path) DO NOTHING
            RETURNING id
        """, (video_path.replace("\\", "/"), os.path.basename(video_path), "Video demo"))
        
        # Lấy kết quả của câu INSERT (row chứa video_id nếu insert thành công)
        row = cur.fetchone()
        if row:
            video_id = row[0]  # Video mới được chèn, lấy video_id
        else:
            # Nếu video đã tồn tại (không insert), truy vấn để lấy video_id
            cur.execute("SELECT id FROM videos WHERE video_path=%s", (video_path.replace("\\", "/"),))
            row = cur.fetchone()
            if row:
                video_id = row[0]  # Lấy video_id từ bản ghi hiện có
            else:
                print(f"❌ Không tìm thấy video {video_path}, bỏ qua.")
                continue

        # Duyệt từng file trong thư mục frame
        for frame_file in os.listdir(frame_dir_path):
            # Chỉ xử lý các file hình ảnh (.jpg, .jpeg, .png)
            if not frame_file.endswith((".jpg", ".jpeg", ".png")):
                continue
            # Tạo đường dẫn đầy đủ tới frame, đảm bảo dùng dấu "/"
            frame_path = os.path.join(frame_dir_path, frame_file).replace("\\", "/")

            # Trích xuất chỉ số frame từ tên file (phần trước đuôi mở rộng)
            try:
                frame_idx = int(os.path.splitext(frame_file)[0])
            except ValueError:
                # Bỏ qua nếu tên file không chuyển được thành số (frame_idx)
                continue

            # Lấy thông tin mapping từ file CSV dựa trên frame_idx
            row = mapping_df[mapping_df["n"] == frame_idx]
            if row.empty:
                # Bỏ qua nếu không tìm thấy thông tin mapping cho frame_idx
                continue

            # Lấy pts_time (thời gian frame) và fps từ file CSV
            pts_time = float(row["pts_time"].values[0])
            fps = int(row["fps"].values[0])

            # Mã hóa frame thành vector đặc trưng bằng hàm encode_image (dùng CLIP)
            emb = encode_image(frame_path)
            if emb is None:
                # Bỏ qua nếu mã hóa frame thất bại
                continue

            # Chèn vector đặc trưng vào Milvus và lấy milvus_id
            milvus_id = insert_to_milvus(frame_path, emb)
            if milvus_id is None:
                # Bỏ qua nếu chèn vào Milvus thất bại
                continue

            # Chèn metadata của frame vào PostgreSQL
            insert_to_postgres(video_id, frame_path, pts_time, frame_idx, fps, milvus_id)

        # Commit giao dịch PostgreSQL sau khi xử lý xong mỗi video
        conn.commit()
        print(f"✅ Indexed {key_frame_dir}")

# ================== Chạy index ==================
# Gọi hàm để bắt đầu pipeline lập chỉ mục video
index_videos()

✅ Indexed L21_V001
✅ Indexed L21_V002
✅ Indexed L21_V003


# Cell 7 – Index incremental (chỉ data mới)

In [ ]:
def index_new_data():
    for key_frame_dir in os.listdir(FRAME_DIR):
        frame_dir_path = os.path.join(FRAME_DIR, key_frame_dir)
        csv_path = os.path.join(CSV_DIR, f"{key_frame_dir}.csv")
        video_path = os.path.join(VIDEO_DIR, f"{key_frame_dir}.mp4")

        if not (os.path.isdir(frame_dir_path) and os.path.exists(video_path) and os.path.exists(csv_path)):
            continue

        mapping_df = pd.read_csv(csv_path)

        # Lấy video_id
        cur.execute("SELECT id FROM videos WHERE video_path=%s", (video_path,))
        video_id = cur.fetchone()
        if not video_id:
            continue
        video_id = video_id[0]

        # Check frame mới
        for frame_file in os.listdir(frame_dir_path):
            if not frame_file.endswith((".jpg",".jpeg",".png")):
                continue
            frame_path = os.path.join(frame_dir_path, frame_file).replace("\\","/")

            cur.execute("SELECT 1 FROM frame_mappings WHERE frame_path=%s", (frame_path,))
            if cur.fetchone():  # đã có thì bỏ qua
                continue

            try:
                frame_idx = int(os.path.splitext(frame_file)[0])
            except:
                continue

            row = mapping_df[mapping_df["n"] == frame_idx]
            if row.empty:
                continue

            pts_time = float(row["pts_time"].values[0])
            fps = int(row["fps"].values[0])

            # Encode
            emb = encode_image(frame_path)
            if emb is None:
                continue

            # Insert Milvus + Postgres
            milvus_id = insert_to_milvus(frame_path, emb)
            insert_to_postgres(video_id, frame_path, pts_time, frame_idx, fps, milvus_id)

        conn.commit()
        print(f"✅ Indexed new frames for {key_frame_dir}")


# Với cấu trúc này bạn có thể:

- Chạy Cell 2 + 3 chỉ 1 lần duy nhất để khởi tạo DB và Collection.

- Sau đó dùng Cell 6 để index tất cả data cũ.

- Về sau chỉ cần chạy Cell 7 để nạp thêm data mới vào, không ảnh hưởng dữ liệu cũ.

# cell 8

Đảm bảo chạy Cell 4 trước Cell 8

In [9]:
import pandas as pd
import os
import torch.nn.functional as F
from deep_translator import GoogleTranslator

def time_to_seconds(time_str):
    """
    Chuyển đổi thời gian dạng mm:ss thành giây.

    Args:
        time_str (str): Thời gian dạng mm:ss (ví dụ: "4:41").

    Returns:
        float: Số giây hoặc None nếu lỗi.
    """
    try:
        minutes, seconds = map(int, time_str.split(':'))
        return minutes * 60 + seconds
    except ValueError:
        print(f"❌ Lỗi định dạng thời gian: {time_str}")
        return None

def get_frame_idx_from_time(video_path, time_start, time_end, csv_dir):
    """
    Tìm frame_idx gần nhất với thời gian trung bình của khoảng [time_start, time_end].

    Args:
        video_path (str): Đường dẫn tới file video.
        time_start (str): Thời gian bắt đầu (mm:ss).
        time_end (str): Thời gian kết thúc (mm:ss).
        csv_dir (str): Thư mục chứa file CSV.

    Returns:
        int or None: frame_idx gần nhất hoặc None nếu lỗi.
    """
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    csv_path = os.path.join(csv_dir, f"{video_name}.csv")

    if not os.path.exists(csv_path):
        print(f"❌ Không tìm thấy file CSV: {csv_path}")
        return None

    try:
        df = pd.read_csv(csv_path)
        if not all(col in df.columns for col in ['pts_time', 'frame_idx', 'fps']):
            print(f"❌ File CSV {csv_path} thiếu cột cần thiết")
            return None
    except Exception as e:
        print(f"❌ Lỗi đọc CSV {csv_path}: {e}")
        return None

    start_sec = time_to_seconds(time_start)
    end_sec = time_to_seconds(time_end)
    if start_sec is None or end_sec is None:
        return None

    target_time = (start_sec + end_sec) / 2
    df['time_diff'] = abs(df['pts_time'] - target_time)
    closest_row = df.loc[df['time_diff'].idxmin()]
    return int(closest_row['frame_idx'])

def format_search_results(search_results, csv_dir):
    """
    Định dạng kết quả từ search_videos_by_text, nhóm theo video_path để mỗi video chỉ in path một lần.

    Args:
        search_results (list): Kết quả từ search_videos_by_text (danh sách dict với video_path, title, time_ranges).
        csv_dir (str): Thư mục chứa file CSV.

    Returns:
        dict: Dictionary với key là video_path, value là tuple (video_name, list of (frame_idx, time_start, time_end)).
    """
    grouped_results = {}
    for res in search_results:
        video_path = res['video_path']
        video_name = os.path.splitext(os.path.basename(video_path))[0]
        if video_path not in grouped_results:
            grouped_results[video_path] = (video_name, [])
        
        for time_start, time_end in res['time_ranges']:
            frame_idx = get_frame_idx_from_time(video_path, time_start, time_end, csv_dir)
            if frame_idx is not None:
                grouped_results[video_path][1].append((frame_idx, time_start, time_end))
    
    return grouped_results

def encode_text(text):
    """
    Mã hóa văn bản thành vector đặc trưng bằng mô hình CLIP.

    Args:
        text (str): Chuỗi văn bản truy vấn.

    Returns:
        numpy.ndarray or None: Vector đặc trưng của văn bản hoặc None nếu lỗi.
    """
    try:
        inputs = processor(
            text=[text],
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=77
        ).to(device)
        with torch.no_grad():
            text_features = model.get_text_features(**inputs)
        return F.normalize(text_features, p=2, dim=1)[0].cpu().numpy()
    except Exception as e:
        print(f"❌ Lỗi encode text: {e}")
        return None

def format_time(seconds):
    """
    Chuyển đổi thời gian (giây) thành định dạng mm:ss.

    Args:
        seconds (float): Thời gian tính bằng giây.

    Returns:
        str: Chuỗi thời gian định dạng mm:ss.
    """
    m = int(seconds // 60)
    s = int(seconds % 60)
    return f"{m}:{s:02d}"

def group_timestamps(timestamps, gap_threshold=10.0):
    """
    Gom các timestamp gần nhau thành các khoảng thời gian.

    Args:
        timestamps (list): Danh sách thời gian (giây) cần nhóm.
        gap_threshold (float): Khoảng cách tối đa giữa hai timestamp để xem là cùng nhóm.

    Returns:
        list: Danh sách các tuple (start, end) biểu thị khoảng thời gian.
    """
    if not timestamps:
        return []
    
    timestamps = sorted(timestamps)
    ranges = []
    start = timestamps[0]
    end = timestamps[0]
    for t in timestamps[1:]:
        if t - end <= gap_threshold:
            end = t
        else:
            ranges.append((start, end))
            start = t
            end = t
    ranges.append((start, end))
    return ranges

def search_videos_by_text(text_query, top_k=10, gap_threshold=10.0):
    """
    Tìm kiếm video dựa trên truy vấn văn bản, trả về danh sách video với khoảng thời gian liên quan.

    Args:
        text_query (str): Văn bản truy vấn.
        top_k (int): Số lượng kết quả tối đa trả về từ Milvus.
        gap_threshold (float): Ngưỡng thời gian để nhóm các timestamp.

    Returns:
        list: Danh sách các video với đường dẫn, tiêu đề và các khoảng thời gian.
    """
    text_emb = encode_text(text_query)
    if text_emb is None:
        return []

    results = client.search(
        collection_name="text_image_video_collection",
        data=[text_emb],
        limit=top_k,
        output_fields=["frame_path"]
    )

    output = []
    for hit in results[0]:
        frame_path = hit["entity"]["frame_path"]
        cur.execute("""
            SELECT v.video_path, v.title, fm.pts_time
            FROM frame_mappings fm
            JOIN videos v ON fm.video_id = v.id
            WHERE fm.frame_path=%s
        """, (frame_path,))
        row = cur.fetchone()
        if row:
            video_path, title, pts_time = row
            output.append((video_path, title, pts_time))

    grouped_results = {}
    for video_path, title, pts_time in output:
        if video_path not in grouped_results:
            grouped_results[video_path] = {"title": title, "timestamps": []}
        grouped_results[video_path]["timestamps"].append(pts_time)

    final_results = []
    for video_path, data in grouped_results.items():
        time_ranges = group_timestamps(data["timestamps"], gap_threshold=gap_threshold)
        final_results.append({
            "video_path": video_path,
            "title": data["title"],
            "time_ranges": [(format_time(s), format_time(e)) for s, e in time_ranges]
        })

    return final_results



import os
import pandas as pd

def keyframe_path_from_frame_idx(video_path, frame_idx, csv_dir, frame_root):
    """
    Map frame_idx -> n (theo CSV) rồi tìm file ảnh keyframe tương ứng.
    Trả về path ảnh hoặc None nếu không tìm thấy.
    """
    video_name = os.path.splitext(os.path.basename(video_path))[0]
    csv_path   = os.path.join(csv_dir, f"{video_name}.csv")
    dir_path   = os.path.join(frame_root, video_name)   # ✅ sửa: bỏ group_name trung gian

    if not os.path.exists(csv_path):
        return None
    try:
        df = pd.read_csv(csv_path)
    except Exception:
        return None

    # Cần cột 'n' và 'frame_idx'
    if not all(c in df.columns for c in ["n", "frame_idx"]):
        return None

    # Tìm hàng có frame_idx gần nhất với frame_idx yêu cầu
    idx = (df["frame_idx"] - int(frame_idx)).abs().idxmin()
    n_val = int(df.loc[idx, "n"])

    # Thử các biến thể tên file
    candidates = []
    for stem in [n_val, f"{n_val:03d}", f"{n_val:04d}", frame_idx, f"{int(frame_idx):03d}", f"{int(frame_idx):04d}"]:
        for ext in ("jpg", "jpeg", "png"):
            candidates.append(os.path.join(dir_path, f"{stem}.{ext}"))

    for p in candidates:
        if os.path.exists(p):
            return p

    return None


## dịch từ việt sang anh để queey nè

In [4]:

# ================== Demo ==================
# Văn bản cần dịch
text_vi = "xe cứu hỏa"
result = GoogleTranslator(source='vi', target='en').translate(text_vi)

# Tìm kiếm video bằng hàm gốc
query = result  # Ví dụ:
search_results = search_videos_by_text(query, top_k=20, gap_threshold=15.0)

# Chuyển đổi kết quả sang định dạng yêu cầu
results = format_search_results(search_results, csv_dir=CSV_DIR)

# In kết quả
print(f"📌 Kết quả tìm kiếm cho: \"{query}\"")
for video_path, (video_name, frames) in results.items():
    print(f"\n📂 Path: {video_path}")
    for frame_idx, time_start, time_end in frames:
        print(f"{video_name}, Frame Idx: {frame_idx} 👉 Xuất hiện từ {time_start} đến {time_end}")

📌 Kết quả tìm kiếm cho: "fire truck"

📂 Path: D:/Big_project_2025/Video_Similarity_Search/data/video/L21_V001.mp4
L21_V001, Frame Idx: 20235 👉 Xuất hiện từ 11:11 đến 11:18
L21_V001, Frame Idx: 27908 👉 Xuất hiện từ 15:30 đến 15:30
L21_V001, Frame Idx: 36318 👉 Xuất hiện từ 20:10 đến 20:10
L21_V001, Frame Idx: 37167 👉 Xuất hiện từ 20:29 đến 20:45

📂 Path: D:/Big_project_2025/Video_Similarity_Search/data/video/L21_V002.mp4
L21_V002, Frame Idx: 12045 👉 Xuất hiện từ 6:26 đến 6:58
L21_V002, Frame Idx: 13083 👉 Xuất hiện từ 7:16 đến 7:16
L21_V002, Frame Idx: 14933 👉 Xuất hiện từ 8:12 đến 8:21

📂 Path: D:/Big_project_2025/Video_Similarity_Search/data/video/L21_V003.mp4
L21_V003, Frame Idx: 12811 👉 Xuất hiện từ 8:32 đến 8:38
L21_V003, Frame Idx: 19380 👉 Xuất hiện từ 12:55 đến 12:55


In [ ]:
from deep_translator import GoogleTranslator
import os

# ---------------- Demo ----------------
# Văn bản tiếng Việt cần tìm
text_vi = "nhiều con cá đang bơi"
# Dịch sang tiếng Anh để match với CLIP
text_en = GoogleTranslator(source='vi', target='en').translate(text_vi)

# Tìm kiếm video dựa trên truy vấn văn bản
search_results = search_videos_by_text(text_en, top_k=20, gap_threshold=15.0)

# Chuyển đổi kết quả sang định dạng video_path -> (video_name, list of (frame_idx, time_start, time_end))
results = format_search_results(search_results, csv_dir=CSV_DIR)

# In kết quả
print(f"Kết quả tìm kiếm cho: \"{text_en}\"")
for video_path, (video_name, frames) in results.items():
    print(f"\nVideo: {video_name}")
    print(f"Path: {video_path}")

    for frame_idx, time_start, time_end in frames:
        img_path = keyframe_path_from_frame_idx(
            video_path=video_path,
            frame_idx=frame_idx,
            csv_dir=CSV_DIR,
            frame_root=FRAME_DIR
        )
        img_path_str = img_path if img_path else "⚠️ Không tìm thấy ảnh"
        print(f"Frame Idx: {frame_idx} -- Xuất hiện từ {time_start} đến {time_end} -- Path: {img_path_str}")

Kết quả tìm kiếm cho: "Many fish are swimming"

Video: L21_V001
Path: D:/Big_project_2025/Video_Similarity_Search/data/video/L21_V001.mp4
Frame Idx: 10868 -- Xuất hiện từ 6:02 đến 6:02 -- Path: D:\Big_project_2025\Video_Similarity_Search\data\key_frame\L21_V001\087.jpg
Frame Idx: 31560 -- Xuất hiện từ 17:11 đến 17:49 -- Path: D:\Big_project_2025\Video_Similarity_Search\data\key_frame\L21_V001\253.jpg
Frame Idx: 32622 -- Xuất hiện từ 18:07 đến 18:11 -- Path: D:\Big_project_2025\Video_Similarity_Search\data\key_frame\L21_V001\261.jpg
Frame Idx: 34961 -- Xuất hiện từ 19:22 đến 19:31 -- Path: D:\Big_project_2025\Video_Similarity_Search\data\key_frame\L21_V001\282.jpg

Video: L21_V003
Path: D:/Big_project_2025/Video_Similarity_Search/data/video/L21_V003.mp4
Frame Idx: 5970 -- Xuất hiện từ 3:51 đến 4:07 -- Path: D:\Big_project_2025\Video_Similarity_Search\data\key_frame\L21_V003\060.jpg
Frame Idx: 9060 -- Xuất hiện từ 5:57 đến 6:06 -- Path: D:\Big_project_2025\Video_Similarity_Search\data\ke

: 

## 📝 Phân tích lại pipeline

1. **Video → Frame → Embedding**

   * Video được chia thành **các keyframe**.
   * Mỗi frame có metadata được lưu vào CSV/DB:

     * **n**: số thứ tự keyframe.
     * **pts\_time**: thời điểm xuất hiện frame trong video (tính bằng giây).
     * **fps**: frame per second (ví dụ: 30 fps).
     * **frame\_idx**: chỉ số frame trong toàn video (fps × thời gian).

   👉 Ví dụ:

   ```
   n=2, pts_time=3.0, fps=30.0, frame_idx=90
   ```

   nghĩa là frame số 90 (tại giây thứ 3.0) được chọn làm keyframe thứ 2.

2. **Index & Lưu trữ**

   * Embedding của từng frame được tính bằng CLIP.
   * Vector này được lưu vào Milvus cùng metadata: `{frame_path, video_id, pts_time, frame_idx}`.

3. **Tìm kiếm (Text → Embedding → Milvus)**

   * Người dùng nhập text (VD: `"fish swimming"`).
   * Text được encode thành vector bằng CLIP.
   * Milvus tìm trong vector database → trả về top-K frame gần nhất.
   * PostgreSQL/CSV được query theo **frame\_idx hoặc pts\_time** để biết frame này nằm ở giây nào, thuộc video nào.
   * Kết quả cuối cùng: **video + khoảng thời gian chứa frame**.



# 🔎 Ý nghĩa của bảng `n, pts_time, fps, frame_idx`

## 1. Nguyên tắc cơ bản

* Trong file CSV có (ví dụ `L21_V001.csv`):

  ```
  n, pts_time, fps, frame_idx
  1, 0.0, 30.0, 0
  2, 3.0, 30.0, 90
  3, 10.0, 30.0, 300
  ...
  ```
* Các thông tin quan trọng:

  * `pts_time`: thời gian (tính bằng giây).
  * `fps`: số khung hình mỗi giây (frames per second).
  * `frame_idx`: chỉ số frame tương ứng với thời gian đó.

👉 Công thức cơ bản:

$$
frame\_idx = time\_in\_seconds \times fps
$$

---

## 2. Ví dụ với **Frame Idx = 8820**

* Giả sử video có **fps = 30**.
* Tính ngược lại thời gian:

  $$
  time\_in\_seconds = \frac{frame\_idx}{fps} = \frac{8820}{30} = 294\ (giây)
  $$
* 294 giây = 4 phút 54 giây (≈ **4:54**).

Trong log của bạn, hệ thống hiển thị:

```
Xuất hiện từ 4:41 đến 5:11
```

👉 Lý do có **khoảng \[4:41 – 5:11]**:

* Khi search, bạn lấy nhiều frame gần nhau (theo pts\_time).
* Sau đó dùng hàm `group_timestamps` để gom lại thành **1 khoảng liên tục** (cách nhau ≤ 10s).
* Do đó frame 8820 (tính toán ra \~4:54) nằm trong **cluster timestamps** từ \~4:41 (281s) → \~5:11 (311s).

---



✅ **Kết luận**

* **Frame idx (8820)** tính được từ:

  $$
  frame\_idx = time \times fps
  $$
* **Khoảng thời gian \[4:41 – 5:11]** được tạo ra do  **gom các timestamps gần nhau thành 1 cluster**, trong đó 8820 nằm giữa.